In [1]:
import ffn
import numpy as np
import pandas as pd
from datetime import datetime
import pickle

In [2]:
from inspect import getmembers, isfunction
import all_strategies

In [3]:
# 計算 MaxDD
def DrawDownAnalysis(cumRet):
    dd_series = ffn.core.to_drawdown_series(cumRet)
    dd_details = ffn.core.drawdown_details(dd_series)
    return dd_details['drawdown'].min(), dd_details['days'].max()

In [4]:
# 利用策略產生的持有部位資訊，計算底下四個指標來判斷投資績效
# sharpe ratio: 判斷報酬的好壞跟穩定度，數值越大越好
# maxdd: maximum drawdown, 最糟糕的狀況會賠幾 %
# maxddd: maximum drawdown duration, 低於上一次最高報酬的天數
# cumRet[-1]: 最後賺的 % 數
def indicators(df):
    dailyRet = df['Close'].pct_change()
    excessRet = (dailyRet - 0.04/252)[df['positions'] == 1]
    SharpeRatio = np.sqrt(252.0)*np.mean(excessRet)/np.std(excessRet)

    cumRet = np.cumprod(1+excessRet)

    maxdd, maxddd = DrawDownAnalysis(cumRet)

    return SharpeRatio, maxdd, maxddd, cumRet[-1]

In [5]:
def apply_strategy(strategy, df):
    return strategy(df)

In [6]:
# 讀出預先下載好的股價資料
with open('../data/twstockdata', 'rb') as f:
    data = pickle.load(file=f)

# 計算各支股票的回測結果
results = []

strategies = [member[1] for member in getmembers(all_strategies) if isfunction(member[1])]

for symbol in data:
    for strategy in strategies:
        try:
            apply_strategy(strategy, data[symbol])
            if np.all(data[symbol]['signals']==0):
                print("Symbol:", symbol, "使用", strategy.__name__, "策略沒有出現買賣訊號。")
                continue
            SharpeRatio, maxdd, maxddd, finalRet = indicators(data[symbol])
            days = (data[symbol].index[-1] - data[symbol].index[0]).days
            results.append((SharpeRatio, maxdd, maxddd, finalRet, days,
                            data[symbol][data[symbol]['signals'] > 0]['signals'].sum(), symbol, strategy.__name__))
        except Exception as e:
            print("Error occurs at symbol:", symbol, "Strategy:", strategy.__name__, "==>", e.args)


results_df = pd.DataFrame(results, columns=['sharpe','MaxDrawDown','MaxDrawDownDuration','returns','days', 'entries','symbol','strategy'])

Symbol: 2330.tw 使用 BBands_strategy 策略沒有出現買賣訊號。
Symbol: 2330.tw 使用 LG_minus3_CL 策略沒有出現買賣訊號。
Symbol: 2330.tw 使用 RSI_7030_strategy 策略沒有出現買賣訊號。
Symbol: 2330.tw 使用 RSI_8020_strategy 策略沒有出現買賣訊號。
Symbol: 2330.tw 使用 team2_strategy 策略沒有出現買賣訊號。
Symbol: 2330.tw 使用 第一組_strategy 策略沒有出現買賣訊號。
Symbol: 2330.tw 使用 第六組_strategy 策略沒有出現買賣訊號。
Symbol: 2330.tw 使用 財運滾滾來_strategy 策略沒有出現買賣訊號。
Symbol: 2317.tw 使用 BBands_strategy 策略沒有出現買賣訊號。
Symbol: 2317.tw 使用 LG_minus3_CL 策略沒有出現買賣訊號。
Symbol: 2317.tw 使用 RSI_7030_strategy 策略沒有出現買賣訊號。
Symbol: 2317.tw 使用 RSI_8020_strategy 策略沒有出現買賣訊號。
Symbol: 2317.tw 使用 team2_strategy 策略沒有出現買賣訊號。
Symbol: 2317.tw 使用 第一組_strategy 策略沒有出現買賣訊號。
Symbol: 2317.tw 使用 第六組_strategy 策略沒有出現買賣訊號。
Symbol: 2317.tw 使用 財運滾滾來_strategy 策略沒有出現買賣訊號。
Symbol: 2412.tw 使用 BBands_strategy 策略沒有出現買賣訊號。
Symbol: 2412.tw 使用 LG_minus3_CL 策略沒有出現買賣訊號。
Symbol: 2412.tw 使用 RSI_7030_strategy 策略沒有出現買賣訊號。
Symbol: 2412.tw 使用 RSI_8020_strategy 策略沒有出現買賣訊號。
Symbol: 2412.tw 使用 team2_strategy 策略沒有出現買賣訊號。
Symbol: 2412.tw 使用 第一組_stra

In [7]:
# Sorted by MaxDrawDown:
results_df.sort_values('MaxDrawDown',ascending=False).head()

,sharpe,MaxDrawDown,MaxDrawDownDuration,returns,days,entries,symbol,strategy
1,0.292448,-0.119232,350,1.038032,871,11,2330.tw,Breakout_strategy
6,1.380548,-0.124380,331,1.347783,871,19,2317.tw,Best_strategy
5,0.966300,-0.124734,65,1.184325,871,8,2330.tw,大盜韓不住_strategy
13,0.052526,-0.127362,643,0.998061,871,7,2412.tw,Breakout_strategy
12,-0.268606,-0.141440,640,0.951341,871,25,2412.tw,Best_strategy


In [8]:
# Sorted by returns:
results_df.sort_values('returns',ascending=False).head()

,sharpe,MaxDrawDown,MaxDrawDownDuration,returns,days,entries,symbol,strategy
23,2.115631,-0.563141,261,8.056075,871,50,3406.tw,中山南拳寶寶_strategy
21,1.871727,-0.558185,261,7.810577,871,11,3406.tw,Team3_strategy
20,1.656400,-0.612821,261,6.513619,871,1,3406.tw,JuianJuian4715_strategy
18,2.428073,-0.315184,252,5.484400,871,23,3406.tw,Best_strategy
19,2.665233,-0.300154,252,4.834767,871,11,3406.tw,Breakout_strategy


In [ ]:
# Sorted by sharpe:
results_df.sort_values('sharpe',ascending=False).head()

In [ ]:
# Sorted by MaxDrawDownDuration:
results_df.sort_values('MaxDrawDownDuration',ascending=True).head()